# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119814e+02     1.572613e+00
 * time: 0.08644390106201172
     1     1.133801e+01     8.922536e-01
 * time: 0.9602398872375488
     2    -1.154351e+01     1.028654e+00
 * time: 1.098404884338379
     3    -3.405442e+01     8.140529e-01
 * time: 1.2304539680480957
     4    -4.755478e+01     5.995804e-01
 * time: 1.3747549057006836
     5    -5.685753e+01     2.212930e-01
 * time: 1.5166139602661133
     6    -5.970123e+01     1.734233e-01
 * time: 1.6183350086212158
     7    -6.086823e+01     7.230666e-02
 * time: 1.7275569438934326
     8    -6.134162e+01     4.020746e-02
 * time: 1.8291888236999512
     9    -6.159725e+01     5.118688e-02
 * time: 1.9384210109710693
    10    -6.179457e+01     2.893343e-02
 * time: 2.0481009483337402
    11    -6.195027e+01     2.324987e-02
 * time: 2.151132822036743
    12    -6.204870e+01     1.857974e-02
 * time: 2.260293960571289
    13    -6.210062e+01     1.484578e-02
 * time: 2.364063978195

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671084
    AtomicLocal         -18.8557690
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336826

    total               -62.261666459285
